In [1]:
from __future__ import annotations

import cProfile
import pstats
from asyncio import sleep

In [2]:
from py_research.db import (
    DataSource,
    TableMap,
    SubTableMap,
    SubMap,
    SelIndex,
    DataBase
)

In [3]:
from schema import (
    Search,
    Project,
    Task,
    User,
    Membership,
    Organization,
    SearchResult,
    TestSchema,
)

In [4]:
from typing import Any


async def _dummy_loader(obj: dict[str, Any]) -> dict:
    await sleep(0.2)
    return {
        'name': obj["name"],
        'role': obj["role"],
        'age': obj["age"] + 1
    }

In [5]:
data_source = DataSource(
    target=Search,
    push={
        "resultCount": Search.result_count,
        "search": Search.term,
        "results": SubTableMap(
            target=Search.results,
            rel_map=TableMap(push={SearchResult.score}),
            push={
                "project_name": Project.name,
                "project_start": Project.start,
                "project_end": Project.end,
                "project_status": Project.status,
                "tasks": SubTableMap(
                    target=Project.tasks,
                    push={
                        "task_name": Task.name,
                        "task_assignees": SubTableMap(
                            target=Task.assignees,
                            push=User.name,
                            match_by=[User.name],
                        ),
                        "task_status": Task.status,
                    },
                ),
                "members": SubTableMap(
                    target=Project.members,
                    loader=_dummy_loader,
                    push={User.name, User.age},
                    match_by=[User.name],
                    rel_map=TableMap(
                        push={
                            Membership.role,
                        },
                    ),
                ),
            },
            pull={
                Project.org: SubMap(
                    push={
                        "organization_name": Organization.name,
                        "organization_address": Organization.address,
                        "organization_city": Organization.city,
                        "organization_countries": Organization.countries,
                    },
                    match_by=[Organization.name, Organization.city],
                ),
                Project.number: SelIndex()
            },
        ),
    },
)

In [6]:
from datetime import date


data = {
    "resultCount": 3,
    "search": "test",
    "results": [
        {
            "score": 0.9,
            "project_name": "baking cake",
            "project_start": date(2020, 1, 1),
            "project_end": date(2020, 1, 4),
            "project_status": "done",
            "organization_name": "Bakery",
            "organization_address": "Main Street 1",
            "organization_city": "Bakerville",
            "organization_countries": ["USA", "Mexico", "Canada"],
            "tasks": [
                {
                    "task_name": "task1",
                    "task_assignees": ["John"],
                    "task_status": "todo",
                },
                {
                    "task_name": "task2",
                    "task_assignees": ["John"],
                    "task_status": "todo",
                },
                {
                    "task_name": "task3",
                    "task_assignees": ["Jane"],
                    "task_status": "done",
                },
            ],
            "members": [
                {"name": "John", "role": "baker", "age": 30},
                {"name": "John", "role": "manager", "age": 40},
            ],
        },
        {
            "score": 0.8,
            "project_name": "cleaning shoes",
            "project_start": date(2020, 1, 2),
            "project_end": date(2020, 1, 5),
            "project_status": "done",
            "organization_name": "Shoe Shop",
            "organization_address": "Main Street 2",
            "organization_city": "Shoetown",
            "organization_countries": ["USA", "Mexico"],
            "tasks": [
                {
                    "task_name": "task4",
                    "task_assignees": ["John"],
                    "task_status": "todo",
                },
                {
                    "task_name": "task5",
                    "task_assignees": ["Jane"],
                    "task_status": "todo",
                },
            ],
            "members": [
                {"name": "John", "role": "cleaner", "age": 25},
                {"name": "Jane", "role": "manager", "age": 35},
            ],
        },
        {
            "score": 0.7,
            "project_name": "fixing cars",
            "project_start": date(2020, 1, 3),
            "project_end": date(2020, 1, 6),
            "project_status": "started",
            "organization_name": "Car Shop",
            "organization_address": "Main Street 3",
            "organization_city": "Cartown",
            "organization_countries": ["USA", "Canada"],
            "tasks": [
                {
                    "task_name": "task6",
                    "task_assignees": ["John"],
                    "task_status": "todo",
                }
            ],
            "members": [
                {"name": "John", "role": "mechanic", "age": 45},
                {"name": "Jane", "role": "manager", "age": 55},
                {"name": "Jack", "role": "manager", "age": 65},
            ],
        },
    ],
}

In [7]:
from pathlib import Path
import warnings

with warnings.catch_warnings():
  warnings.simplefilter("error")
  with cProfile.Profile() as pr:
    db = DataBase(
      schema=TestSchema,
      # url=Path("./test.xlsx")
    )
    rec = await data_source.load([data], db=db)

    stats = pstats.Stats(pr)

Post-loading: `Assignment`: 100%|██████████| 6/6 [00:00<00:00, 204.69it/s]


In [8]:
db[User].extract()

╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ c:\Users\lworm\Repos\py-research\.venv\Lib\site-packages\sqlalchemy\engine\base.py:1964 in       │
│ _exec_single_context                                                                             │
│                                                                                                  │
│   1961 │   │   │   │   │   │   │   evt_handled = True                                            │
│   1962 │   │   │   │   │   │   │   break                                                         │
│   1963 │   │   │   │   if not evt_handled:                                                       │
│ ❱ 1964 │   │   │   │   │   self.dialect.do_execute(                                              │
│   1965 │   │   │   │   │   │   cursor, str_statement, effective_parameters, context              │
│   1966 │   │   │   │   │   )                                                                     │
│   1967                                                                                           │
│                                                                                                  │
│ c:\Users\lworm\Repos\py-research\.venv\Lib\site-packages\sqlalchemy\engine\default.py:942 in     │
│ do_execute                                                                                       │
│                                                                                                  │
│    939 │   │   cursor.executemany(statement, parameters)                                         │
│    940 │                                                                                         │
│    941 │   def do_execute(self, cursor, statement, parameters, context=None):                    │
│ ❱  942 │   │   cursor.execute(statement, parameters)                                             │
│    943 │                                                                                         │
│    944 │   def do_execute_no_params(self, cursor, statement, context=None):                      │
│    945 │   │   cursor.execute(statement)                                                         │
╰──────────────────────────────────────────────────────────────────────────────────────────────────╯
OperationalError: ambiguous column name: schema.Task.assignees._id

The above exception was the direct cause of the following exception:

╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ in <module>:1                                                                                    │
│                                                                                                  │
│ ❱ 1 db[User].extract()                                                                           │
│   2                                                                                              │
│                                                                                                  │
│ C:\Users\lworm\Repos\py-research\src\py_research\db\databases.py:1095 in extract                 │
│                                                                                                  │
│   1092 │   │   while to_traverse:                                                                │
│   1093 │   │   │   target = to_traverse.pop(list(to_traverse.keys())[0])                         │
│   1094 │   │   │                                                                                 │
│ ❱ 1095 │   │   │   if len(target) == 0:                                                          │
│   1096 │   │   │   │   continue                                                                  │
│   1097 │   │   │                                                                                 │
│   1098 │   │   │   select = Data[Any, Any, Any, Any, Any, Any].select(                           │
│                       

In [ ]:
db[Search.results.x.tasks.x.assignees].df(sort_by=[User.name])

schema.Search.term,schema.Search.results.number,schema.Search.results.tasks._id,_id,name,age
str,i64,str,str,str,i64
"""test""",1,"""eb20fd66-f1a2-48ad-af58-6ded47…","""417a4b38-b2f1-47e3-ac7b-4430a5…","""Jane""",56
"""test""",0,"""9c136895-dcfd-437b-8be9-27ffc8…","""417a4b38-b2f1-47e3-ac7b-4430a5…","""Jane""",56
"""test""",1,"""5376074e-09b3-431e-9395-9ecb33…","""424f2f5c-b929-4288-9120-682c08…","""John""",31
"""test""",0,"""0e0865d9-27fd-4287-99ad-db813a…","""424f2f5c-b929-4288-9120-682c08…","""John""",31
"""test""",2,"""2a077ee0-73e6-496e-8ae6-a8b96f…","""424f2f5c-b929-4288-9120-682c08…","""John""",31
"""test""",0,"""52e7fecd-68b9-4621-bdb6-1259bb…","""424f2f5c-b929-4288-9120-682c08…","""John""",31


In [ ]:
db.describe()


{
    'schema': {
        'package': 'py-research',
        'module': 'schema',
        'object': 'TestSchema',
        'package_version': '^5.1.1.dev152+g2c0bcb5.d20250123',
        'repo': 'https://github.com/cloudlane-one/py-research.git',
        'repo_revision': 'c3fc7fd980af82ec206bbf4eb506865a438a7312',
        'module_path': WindowsPath('exp/schema.py'),
        'module_dirty': False,
        'docs_url': 'https://cloudlane-one.github.io/py-research/'
    },
    'contents': {
        'records': {
            'schema.User': 3,
            'schema.Search': 1,
            'schema.Organization': 3,
            'schema.Project': 3,
            'schema.Task': 6
        },
        'arrays': {'schema.Organization.countries': 7},
        'relations': {'schema.Assignment': 6, 'schema.SearchResult': 3, 'schema.Membership': 7}
    },
    'backend': None
}

In [11]:
nodes, edges = db.to_graph([Search, Project, Organization])

In [12]:
nodes

,node_id,table,schema.Search.term,schema.Search.result_count,schema.Project.number,schema.Project.name,schema.Project.start,schema.Project.end,schema.Project.status,schema.Project.org__id,schema.Organization._id,schema.Organization.name,schema.Organization.address,schema.Organization.city
0,0,schema_Search,test,3.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1,schema_Project,NaN,NaN,0.0,baking cake,2020-01-01,2020-01-04,done,8521317b-6fdb-4c0e-8780-9792d2c672d7,NaN,NaN,NaN,NaN
2,2,schema_Project,NaN,NaN,1.0,cleaning shoes,2020-01-02,2020-01-05,done,deb553a6-6224-4e5c-9ff6-e6b8b1d1d65c,NaN,NaN,NaN,NaN
3,3,schema_Project,NaN,NaN,2.0,fixing cars,2020-01-03,2020-01-06,started,7070210f-4897-482b-83ff-faa5ede3a0f7,NaN,NaN,NaN,NaN
4,4,schema_Organization,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,7070210f-4897-482b-83ff-faa5ede3a0f7,Car Shop,Main Street 3,Cartown
5,5,schema_Organization,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,8521317b-6fdb-4c0e-8780-9792d2c672d7,Bakery,Main Street 1,Bakerville
6,6,schema_Organization,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,deb553a6-6224-4e5c-9ff6-e6b8b1d1d65c,Shoe Shop,Main Street 2,Shoetown


In [13]:
edges

,source,target,ltr,rtl,schema.SearchResult._to_number,schema.SearchResult._id,schema.SearchResult.score,schema.SearchResult._from_term
0,1,5,org__id,None,NaN,NaN,NaN,NaN
1,2,6,org__id,None,NaN,NaN,NaN,NaN
2,3,4,org__id,None,NaN,NaN,NaN,NaN
3,0,3,_to_number,_from_term,2.0,104d65b581,0.7,test
4,0,2,_to_number,_from_term,1.0,11faab1a21,0.8,test
5,0,1,_to_number,_from_term,0.0,14941c52d3,0.9,test


In [ ]:
db[User].values()


[
    User({'_id': '417a4b38-b2f1-47e3-ac7b-4430a510a442', 'name': 'Jane', 'age': 56}),
    User({'_id': '424f2f5c-b929-4288-9120-682c08f97913', 'name': 'John', 'age': 31}),
    User({'_id': 'ad805077-61b0-4ee1-954f-d790286d0fd1', 'name': 'Jack', 'age': 66})
]

In [ ]:
db[Project].df()

number,name,start,end,status,org__id
i64,str,date,date,str,str
0,"""baking cake""",2020-01-01,2020-01-04,"""done""","""8521317b-6fdb-4c0e-8780-9792d2…"
1,"""cleaning shoes""",2020-01-02,2020-01-05,"""done""","""deb553a6-6224-4e5c-9ff6-e6b8b1…"
2,"""fixing cars""",2020-01-03,2020-01-06,"""started""","""7070210f-4897-482b-83ff-faa5ed…"


In [ ]:
db[Search.term].values()

['test']

In [ ]:
list(db[Search.result_count].items())

[('test', 3)]

In [ ]:
list(db[Search.results.x.org].items())


[
    (
        ('test', 0, '8521317b-6fdb-4c0e-8780-9792d2c672d7'),
        Organization({'_id': '8521317b-6fdb-4c0e-8780-9792d2c672d7', 'name': 'Bakery', 'address': 'Main Street 1', 'city': 'Bakerville'})
    ),
    (
        ('test', 1, 'deb553a6-6224-4e5c-9ff6-e6b8b1d1d65c'),
        Organization({'_id': 'deb553a6-6224-4e5c-9ff6-e6b8b1d1d65c', 'name': 'Shoe Shop', 'address': 'Main Street 2', 'city': 'Shoetown'})
    ),
    (
        ('test', 2, '7070210f-4897-482b-83ff-faa5ede3a0f7'),
        Organization({'_id': '7070210f-4897-482b-83ff-faa5ede3a0f7', 'name': 'Car Shop', 'address': 'Main Street 3', 'city': 'Cartown'})
    )
]

In [ ]:
db[Search.results].items()


[
    (
        ('test', 0),
        Project({'number': 0, 'name': 'baking cake', 'start': datetime.date(2020, 1, 1), 'end': datetime.date(2020, 1, 4), 'status': 'done'})
    ),
    (
        ('test', 1),
        Project({'number': 1, 'name': 'cleaning shoes', 'start': datetime.date(2020, 1, 2), 'end': datetime.date(2020, 1, 5), 'status': 'done'})
    ),
    (
        ('test', 2),
        Project({'number': 2, 'name': 'fixing cars', 'start': datetime.date(2020, 1, 3), 'end': datetime.date(2020, 1, 6), 'status': 'started'})
    )
]

In [ ]:
list(db[Search.results]._abs_cols.keys())

['number', 'name', 'start', 'end', 'status', 'org__id']

In [ ]:
db[Search.results].df()

schema.Search.term,number,name,start,end,status,org__id
str,i64,str,date,date,str,str
"""test""",0,"""baking cake""",2020-01-01,2020-01-04,"""done""","""8521317b-6fdb-4c0e-8780-9792d2…"
"""test""",1,"""cleaning shoes""",2020-01-02,2020-01-05,"""done""","""deb553a6-6224-4e5c-9ff6-e6b8b1…"
"""test""",2,"""fixing cars""",2020-01-03,2020-01-06,"""started""","""7070210f-4897-482b-83ff-faa5ed…"


In [ ]:
db[Search.results.x.org].df()

schema.Search.term,schema.Search.results.number,_id,name,address,city
str,i64,str,str,str,str
"""test""",0,"""8521317b-6fdb-4c0e-8780-9792d2…","""Bakery""","""Main Street 1""","""Bakerville"""
"""test""",1,"""deb553a6-6224-4e5c-9ff6-e6b8b1…","""Shoe Shop""","""Main Street 2""","""Shoetown"""
"""test""",2,"""7070210f-4897-482b-83ff-faa5ed…","""Car Shop""","""Main Street 3""","""Cartown"""


In [ ]:
db[Organization].df()

_id,name,address,city
str,str,str,str
"""7070210f-4897-482b-83ff-faa5ed…","""Car Shop""","""Main Street 3""","""Cartown"""
"""8521317b-6fdb-4c0e-8780-9792d2…","""Bakery""","""Main Street 1""","""Bakerville"""
"""deb553a6-6224-4e5c-9ff6-e6b8b1…","""Shoe Shop""","""Main Street 2""","""Shoetown"""


In [ ]:
db[SearchResult._table[SearchResult.score.isin(slice(0.81, 0.9))]].df()

_id,score,_from_term,_to_number
str,f64,str,i64
"""14941c52d3""",0.9,"""test""",0


In [ ]:
import pandas as pd


db[Organization.countries].df(pd.DataFrame)

value
schema.Organization._id              schema.Organization.countries.idx        
7070210f-4897-482b-83ff-faa5ede3a0f7 0                                     USA
                                     1                                  Canada
8521317b-6fdb-4c0e-8780-9792d2c672d7 0                                     USA
                                     1                                  Mexico
                                     2                                  Canada
deb553a6-6224-4e5c-9ff6-e6b8b1d1d65c 0                                     USA
                                     1                                  Mexico

In [ ]:
stats.sort_stats("cumulative")
stats.print_stats("py_research")

         512460 function calls (466434 primitive calls) in 1.199 seconds

   Ordered by: cumulative time
   List reduced from 2294 to 160 due to restriction <'py_research'>

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
       55    0.001    0.000    0.489    0.009 C:\Users\lworm\Repos\py-research\src\py_research\db\datasources.py:696(_load_rec_from_item)
       55    0.002    0.000    0.483    0.009 C:\Users\lworm\Repos\py-research\src\py_research\db\datasources.py:571(_load_record)
       11    0.000    0.000    0.429    0.039 C:\Users\lworm\Repos\py-research\src\py_research\db\datasources.py:841(load)
    37/12    0.002    0.000 

   0.420    0.035 C:\Users\lworm\Repos\py-research\src\py_research\db\datasources.py:664(_load_records)
18842/8850    0.028    0.000    0.343    0.000 C:\Users\lworm\Repos\py-research\src\py_research\caching.py:382(__get__)
       19    0.000    0.000    0.328    0.017 C:\Users\lworm\Repos\py-research\src\py_research\db\databases.py:1553(__ior__)
       19    0.001    0.000    0.327    0.017 C:\Users\lworm\Repos\py-research\src\py_research\db\databases.py:3010(_mutate)
       48    0.001    0.000    0.319    0.007 C:\Users\lworm\Repos\py-research\src\py_research\db\databases.py:742(keys)
       48    0.002    0.000    0.313    0.007 C:\Users\lworm\Repos\py-research\src\py_research\db\databases.py:913(df)
       18    0.001    0.000    0.280    0.016 C:\Users\lworm\Repos\py-research\src\py_research\db\databases.py:3147(_mutate_from_records)
       16    0.000    0.000    0.192    0.012 C:\Users\lworm\Repos\py-research\src\py_research\db\databases.py:2984(_df_to_table)
      308    0.003